# 初始化spark环境

In [5]:
from tensorflow.keras import Model 
from utils_empirical import * 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers


In [9]:
import pandas as pd 
import tensorflow_ranking as tfr 
from scipy.special import kl_div
import numpy as np 

def dim_est_naive(obs_T, obs_C):
    n1,n0 = len(obs_T), len(obs_C)
    return np.mean(obs_T) -np.mean(obs_C), np.sqrt(np.var(obs_T)/n1 + np.var(obs_C) / n0)



import tensorflow_ranking as tfr 
## Some Helper Functions 
def find_cosine(A_vec, B_vec):
    A = np.array([float(i) for i in A_vec])
    B = np.array([float(i) for i in B_vec])
    return float(np.dot(A,B)/(np.linalg.norm(A,2)*np.linalg.norm(B,2)))

def convert_string_to_vector(s):
    res = s.split(",")
    return np.array(res).astype(float)

def find_cosine_string(s1, s2):
    return find_cosine(convert_string_to_vector(s1), convert_string_to_vector(s2))

def mySoftMax(arr):
    num = np.exp(arr)
    denom = np.sum(num)
    return num/denom


def naive_est(res):
    treat_res = [elm[0] for elm in res[0]]
    control_res = [elm[1] for elm in res[0]]
    return np.mean(treat_res) - np.mean(control_res)


def dim_est(obs_T, obs_C):
    n1,n0 = len(obs_T), len(obs_C)
    return np.mean(obs_T) -np.mean(obs_C), np.sqrt(np.var(obs_T)/n1 + np.var(obs_C) / n0)


def point_est(all_treat_array, all_control_array):
    mus_T, mus_C  = all_treat_array[:, 11:21], all_control_array[:,11:21]
    p_T, p_C  = all_treat_array[:, 21:], all_control_array[:,21:]
    return np.mean(np.sum((mus_T * (p_T - p_C)), axis = 1 ))


def naive_dim_estimate(vector_T, vector_C):
    return np.mean(vector_T) - np.mean(vector_C), np.var(vector_T)/len(vector_T) + np.var(vector_C) / len(vector_C)

def is_invertible(matrix):
    return np.linalg.det(matrix) != 0

def debias_estimator(Hfuncs, debias_terms):
    score_functions = Hfuncs - debias_terms 
    undebiased_estimator = np.mean(Hfuncs)
    debiased_estimator = np.mean(score_functions)
    variance_estimator = np.mean((score_functions - debiased_estimator)**2) / len(score_functions)
    return debiased_estimator, variance_estimator, undebiased_estimator 


def dim_est_IPW(obs_T, obs_C, treated_probability, Q):
    n1,n0 = len(obs_T), len(obs_C)
    tau1 = np.sum(obs_T) / (Q*treated_probability)
    tau0 = np.sum(obs_C)/(Q * (1-treated_probability))
    estimate = tau1 - tau0
    # var = np.sum((obs_T - tau1)**2)/ Q/treated_probability**2  + np.sum((obs_C - tau0)**2)/ Q/(1-treated_probability)**2 
    var = (np.sum((obs_T / treated_probability - estimate) ** 2) + np.sum(( - obs_C / (1-treated_probability) - estimate) ** 2)) / Q
    return estimate, var


def aggregate_probability_exposure_examination(probs_matrix, exposure_matrix): 
    aggregate_probs = np.mean(probs_matrix, axis = 0)
    exposure_probs = np.mean(exposure_matrix, axis = 0)
    ## Euclidean distance 
    euc_dist = np.linalg.norm(aggregate_probs - exposure_probs)
    ## NDCG LOSS 
    y_true = tf.ragged.constant(exposure_matrix)
    y_pred = tf.ragged.constant(probs_matrix)
    loss_NDCG = tfr.keras.losses.ApproxNDCGLoss(ragged=True)
    NDCG_loss_result = loss_NDCG(y_true, y_pred).numpy()
    return euc_dist, aggregate_probs, exposure_probs, NDCG_loss_result

def bootstrap_evaluation(probs_mat, expos_mat, B = 100):
    nrows = probs_mat.shape[0]
    kl_div_list, NDCG_list = [], []
    for b in range(B):
        indices = np.random.choice(nrows, size=nrows, replace=True)
        probs_mat_b = probs_mat[indices,:]
        expos_mat_b = expos_mat[indices,:]

        _, agg_probs_b, agg_expos_b, NDCG_b = aggregate_probability_exposure_examination(probs_mat_b, expos_mat_b)
        kl_divergence_b = kl_div(agg_probs_b,agg_expos_b).sum()

        kl_div_list += [kl_divergence_b]
        NDCG_list += [NDCG_b]

    return kl_div_list, NDCG_list 




## Modifying the tensor for 3d input 
class MyModel_multiple(Model):
    def __init__(self, k, num_treats,predictY=False):
        super(MyModel_multiple, self).__init__()
        self.k = k
        self.num_treats = num_treats
        self.groupNames = ['A'] + ['B' + str(i+1) for i in range(self.num_treats)]
        
        self.baseline_logit = Dense( 1, activation = "linear")
        self.logit_dense_layer = {} 
        for g in self.groupNames:
            self.logit_dense_layer[g] = Dense(1, activation = "linear")
        self.common_hidden = Dense(3, activation = "linear")
        self.softmax =tf.keras.activations.softmax
        
        self.predictY = predictY
        self.d5 = Dense(self.k, activation = "relu")
        self.doutcome = Dense(1)
        
        self.d_onehot = tf.keras.layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), k))
        
    
    def call(self, inputs):

        split_structure =  [4] + [1] * self.num_treats + [1]
        splitted_elements = tf.split(inputs, split_structure, axis=2)
        x1 = splitted_elements[0]
        ypredicts = splitted_elements[-1]
        
        
        
        
        ## Step 1: Reshape the input 
        
        reshape_x1 = tf.reshape(x1, (-1, x1.shape[-1]))
        
        ## Step 2: a common hidden layer
        x1_common_hidden = self.common_hidden(reshape_x1)
        
        #x1_common_hidden_3d = tf.reshape(x1_common_hidden, [x1.shape[0],self.k, x1_common_hidden.shape[1]])
        x1_common_hidden_3d = tf.reshape(x1_common_hidden,[-1, k, 3])
        
        ## Baseline logit
        x1_final = self.baseline_logit(x1_common_hidden_3d)
        
        # Get the first element of the second dimension
        # first_element = x1_final[:, 0:1, :]

        # Subtract the first element from every element of the second dimension
        # x1_final = x1_final - first_element

        
        ## Step 3: logit model
        for i in range(self.num_treats):
            w_g = splitted_elements[i + 1]
            xg_hidden = self.logit_dense_layer['B'+str(i+1)](x1_common_hidden_3d)
            x1_final = tf.add(tf.multiply(w_g, xg_hidden), x1_final)
            
        ## Step 4: Softmax
        reshaped_data = tf.squeeze(x1_final, axis=-1)
        softmax_p =  self.softmax(reshaped_data)

        # for g in self.groupNames:
        #     if g != 'A':
        #         w_g = self.treatment_matrix_dict[g]
        #         x = tf.math.add(tf.multiply(w_g, self.logit_dense_layer[g](x1)), x)
        #x = tf.add(tf.multiply(self.treatment_matrix_dict['B1'],  x2_hidden), x1_hidden)

        # x = tf.mul( w, x1_hidden)

        y1 = self.d_onehot(softmax_p)
        
        if self.predictY:
            x5 = self.d5(x1)
            y2 = self.doutcome(tf.multiply(softmax_p, x5))
        else:
            
            ypredicts = splitted_elements[-1]
            ypredicts = tf.squeeze(ypredicts, axis=-1)
            
            y2 = tf.reduce_sum(tf.multiply(softmax_p, ypredicts), axis = 1 )
            y2 = tf.expand_dims(y2, axis=-1)

            
        # ## One hot vector  
        # y1 = softmax_p
        # x5 = self.d5(ypredicts)
        # print("softmax shape", y1.shape)
        
        # ## Outcome 
    
        # y2 = self.doutcome(tf.multiply(y1, x5))
        


        res = tf.concat([softmax_p,y2,softmax_p], axis=1)
        return res

# Define custom loss function
def custom_loss(y_true, y_pred):

    y1_true, y2_true = tf.split(y_true, [K, 1], axis=1)
    _, y2_pred, y1_pred = tf.split(y_pred, [K, 1, K], axis=1)


    loss1 = tf.keras.losses.CategoricalCrossentropy()(y1_true, y1_pred)
    loss2 = tf.keras.losses.MeanSquaredError()(y2_true, y2_pred)
    return loss1 + loss2 
    
    
    
    
## Helper functions 

def debias_estimator_new(Hfuncs, debias_terms,tau_hat):
    psi_functions = Hfuncs - debias_terms 
    undebiased_estimator = np.mean(Hfuncs)
    debiased_estimator = np.mean(psi_functions)
    variance_estimator = np.sum((psi_functions - tau_hat)**2) /len(psi_functions)
    return debiased_estimator, variance_estimator, undebiased_estimator 


def undebias_estimator_new(Hfuncs,tau_hat):
    psi_functions = Hfuncs 
    undebiased_estimator = np.mean(Hfuncs)
    variance_estimator = np.sum((psi_functions - tau_hat)**2) /len(psi_functions)
    
    return undebiased_estimator, variance_estimator 


def is_invertible(matrix):
    return np.linalg.det(matrix) != 0


    
def permute_treatment_dict(J, L):
    perm_dict = {}
    for j in range(J):
        perm_dict[j] = np.random.choice(L+1, 1)
    return perm_dict

## Helper function for cross validation
def generate_indices(n, K):
    ## Split original sample of size n into K sets 
    indices = np.linspace(0, n, K+1, dtype=int)
    return list(zip(indices[:-1], indices[1:]))


def train_test_split(input_data, all_inds, kth_test):
    
    training_ind = [all_inds[i] for i in range(len(all_inds)) if i != kth_test]
    test_start, test_end = all_inds[kth_test]
    if not tf.is_tensor(input_data):
        training_data = np.concatenate([input_data[elm[0]:elm[1]] for elm in training_ind])
    else:
        
        training_data = tf.concat([input_data[elm[0]:elm[1]] for elm in training_ind], axis = 0)
    testing_data = input_data[test_start:test_end]
    return training_data, testing_data 

In [10]:
ith_treatment = 0 
K = 15
k = 15
ith_outcome = 1
L = 1
ith_treat = 0


In [11]:
n_folds=5

In [12]:
# model_memories = {}
# for f in range(n_folds):
#     model_f = MyModel_multiple(K,L)
#     model_f.compile(loss=custom_loss,optimizer=tf.keras.optimizers.Adam())
#     model_memories[f] = model_f

2024-06-18 08:31:31.975994: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-18 08:31:31.976061: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-18 08:31:31.976084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (VM-209-192-centos): /proc/driver/nvidia/version does not exist
2024-06-18 08:31:31.976609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
fromDate = '20240527'
toDate = '20240604'
import datetime
from datetime import date, timedelta
import requests

def date_range(start, end):
    delta = end - start  # as timedelta
    days = [start + timedelta(days=i) for i in range(delta.days + 1)]
    return days

def form_ds(y,m,d):
    yyyy = str(y)
    if m < 10:
        mm = '0' + str(m)
    else:
        mm = str(m)
    if d < 10:
        dd = '0' + str(d)
    else:
        dd = str(d)
    return yyyy+mm+dd
date_range_obj = date_range(date(int(fromDate[0:4]), int(fromDate[4:6]), int(fromDate[6:8])), date(int(toDate[0:4]), int(toDate[4:6]), int(toDate[6:8])))
date_range_p_list = ['p_' + form_ds(d.year,d.month ,d.day) for d in date_range_obj]
date_range_loghour_list = []
for date_p in date_range_p_list:
    for hr in range(24):
        if hr < 10:
            
            date_range_loghour_list += [date_p + '0' +str(hr)]
        else:
            date_range_loghour_list += [date_p + str(hr)]


In [14]:
controlGroupIDs = [92028784]
treatGroupIDs = [92028785]

In [15]:

def compute_value_gradient_subgroup(predict_p_treat, predict_outcome_treat, predict_p_control, predict_outcome_control, J):
    # J: dimension K, indicator function of whether item k belongs to the subgroup
    Ey1 = np.sum(predict_p_treat * predict_outcome_treat * J, axis=1, keepdims=True)
    Ey0 = np.sum(predict_p_control * predict_outcome_control * J, axis=1, keepdims=True)
    dHdtheta0 = predict_p_treat * (predict_outcome_treat * J - Ey1) - predict_p_control * (predict_outcome_control * J - Ey0)
    dHdtheta0 = dHdtheta0[:, 1:]
    dHdtheta1 = predict_p_treat * (predict_outcome_treat * J - Ey1) 
    dHdmu = (predict_p_treat - predict_p_control) * J
    gradient_vector_H = np.concatenate([dHdtheta0, dHdtheta1, dHdmu], axis =1 )
    return gradient_vector_H

In [16]:
cate_groups = ['f4_51到100元','f5_101到500元','f3_21到50元','f1_<=10元']

In [17]:
import pickle
n_folds = 5

In [1]:
## Model training 
for hr in range(len(date_range_loghour_list)):
    print("Start for hour", hr)
    dh = hr
    provider_weixin_experiment.table("", priParts = [date_range_loghour_list[dh]]).createOrReplaceTempView("df_vv_details")
    df_spark = session.sql(f"select * from df_vv_details")
    df_input = df_spark.toPandas()
    print("Finish loading the data")


    ## data-preprocessing for different outcomes 
    df_input['outcome1'] = df_input['max_target_action_cnt']
    df_input['outcome2'] = df_input['bid_']
    ## outcome3 - type1 conversion 
    df_input['outcome3'] = ((df_input['promotion_type']==1 ).astype(float)) * (df_input['max_target_action_cnt'].astype(float))
    ## outcome4 - type2 conversion 
    df_input['outcome4'] = ((df_input['promotion_type']==2 ).astype(float)) * (df_input['max_target_action_cnt'].astype(float))
    ## outcome5 - type1 bid 
    df_input['outcome5'] = ((df_input['promotion_type']==1 ).astype(float)) * (df_input['bid_'].astype(float))
    ## outcome6 - type2 bid 
    df_input['outcome6'] = ((df_input['promotion_type']==2 ).astype(float)) * (df_input['bid_'].astype(float))




    ## data-preprocessing for different outcomes 
    df_input['outcome1_pred'] = df_input['cvr_']
    df_input['outcome2_pred'] = df_input['bid_']
    ## outcome3 - type1 conversion 
    df_input['outcome3_pred'] = ((df_input['promotion_type']==1 ).astype(float)) * (df_input['cvr_'].astype(float))
    ## outcome4 - type2 conversion 
    df_input['outcome4_pred'] = ((df_input['promotion_type']==2).astype(float))  * (df_input['cvr_'].astype(float))
    ## outcome5 - type1 bid 
    df_input['outcome5_pred'] = ((df_input['promotion_type']==1 ).astype(float))  * (df_input['bid_'].astype(float))
    ## outcome6 - type2 bid 
    df_input['outcome6_pred'] = ((df_input['promotion_type']==2 ).astype(float))  * (df_input['bid_'].astype(float))
                                                                                   

    df_sellerside = df_input[df_input['groupid'].isin([92028784,92028785])]

    df_twoside = df_input[~df_input['groupid'].isin([92028784,92028785])]
    df_twoside.to_csv(f"prediction_result_new/twoside_data{hr}.csv")

    exposed_twoside = df_twoside.loc[df_twoside['selected']==1,:]
    model_memories = {}
    for f in range(n_folds):
        model_f = MyModel_multiple(K,L)
        model_f.compile(loss=custom_loss,optimizer=tf.keras.optimizers.Adam())
        model_memories[f] = model_f

    ## Result from ground-truth experiment 
    groundtruth_outcome_dict = {}
    for i in range(6):
        twoside_T, twoside_C = exposed_twoside.loc[exposed_twoside['groupid']==92028783,f"outcome{i+1}"],exposed_twoside.loc[exposed_twoside['groupid'].isin([92028781,92028782]),f"outcome{i+1}"]
        dim_naive_i = dim_est_naive(twoside_T, twoside_C)
        dim_ipw_i = dim_est_IPW(twoside_T,twoside_C, 0.5, len(twoside_T) + len(twoside_C))
        groundtruth_outcome_dict[i] = [dim_naive_i,dim_ipw_i, len(twoside_T), len(twoside_C)]
        with open(f"prediction_result_new/groundtruth_outcome{i+1}_{dh}.pickle", 'wb') as ff:
            pickle.dump( groundtruth_outcome_dict, ff)
    print("Finish writing ground truth for ", dh)

    ## Covariates
    base_bid_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = 'bid_')
    sort_score_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = 'sort_score_')
    cvr_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = 'cvr_')
    ecpm_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = 'ecpm_')
    covariates_table = df_sellerside.loc[:, ['req_id_','quota_level','row_number']].pivot_table(index='req_id_', columns = 'row_number', values= 'quota_level', aggfunc=lambda x: x)
    x_covariates = covariates_table.values
    all_inds = generate_indices(base_bid_table.shape[0], n_folds)

    ## Tables 
    ## Covariates
    outcome_table_dict = {} 
    prediction_table_dict = {} 
    outcome_tensor_dict = {} 
    prediction_tensor_dict = {}
    outcome_vector_dict = {} 
    outcome_vector_tensor_dict = {} 
    for i in range(6):
        outcome_i_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = f"outcome{i+1}")
        outcome_table_dict[i+1] = outcome_i_table

        outcome_i_tensor = tf.convert_to_tensor(outcome_i_table.fillna(0).values, dtype = float)
        outcome_tensor_dict[i+1] = outcome_i_tensor

        pred_i_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = f"outcome{i+1}_pred")
        prediction_table_dict[i+1] = pred_i_table

        pred_i_tensor = tf.convert_to_tensor(pred_i_table.fillna(0).values, dtype = float)
        prediction_tensor_dict[i+1] = pred_i_tensor


        outcome_i_vec = df_sellerside.loc[df_sellerside['selected']==1, ['req_id_', f"outcome{i+1}"]].pivot_table(index='req_id_', values= f"outcome{i+1}")
        outcome_vector_dict[i+1] = outcome_i_vec

        outcome_i_vec_tensor = tf.convert_to_tensor(outcome_i_vec.values, dtype = float)
        outcome_vector_tensor_dict[i+1] = outcome_i_vec_tensor

    ## Exposure indicator 

    is_selected_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = 'selected')
    is_selected_indicator = is_selected_indicator = tf.convert_to_tensor(is_selected_table.fillna(0).values, dtype = float)
    exposure_matrix = is_selected_table.values 
    ## Convert to tensor 
    x_sort_score = tf.convert_to_tensor(sort_score_table.fillna(0).values, dtype = float)
    x_basebid = tf.convert_to_tensor(base_bid_table.fillna(0).values, dtype=float)
    x_ecpm = tf.convert_to_tensor(ecpm_table.fillna(0).values, dtype = float)
    x_cvr = tf.convert_to_tensor(cvr_table.fillna(0).values, dtype = float)
    groupid_table = df_sellerside.pivot_table(index='req_id_', columns = 'row_number', values = 'groupid')
    w_dict = {}
    for i in range(len(treatGroupIDs)):
        w_dict[i] = tf.convert_to_tensor(groupid_table.values == treatGroupIDs[i], dtype = float)
    
    for i in range(6):
        
        ith_outcome = i + 1 
        print("Start outcome for ", ith_outcome)
        inputs_3d = tf.stack([x_basebid, x_sort_score,x_ecpm, x_cvr] + [w_dict[v] for v in range(len(treatGroupIDs))] + [prediction_tensor_dict[ith_outcome]], axis = 2)
        exposure_indicator_outcome = tf.concat([is_selected_indicator, outcome_vector_tensor_dict[ith_outcome]], axis = 1)


        samp_size = exposure_indicator_outcome.shape[0]
        w_all_treat = tf.convert_to_tensor(np.array([[1] * k for _ in range(samp_size)],dtype='float32'))
        w_all_control = tf.convert_to_tensor(np.array([[0] * k for _ in range(samp_size)],dtype='float32'))

        inputs_all_treat_3d = tf.stack([x_basebid, x_sort_score,x_ecpm, x_cvr] +  [w_all_treat if l == ith_treat else w_all_control for l in range(L)]+ [prediction_tensor_dict[ith_outcome]], axis = 2)
        inputs_all_control_3d = tf.stack([x_basebid, x_sort_score,x_ecpm, x_cvr] + [w_all_control for l in range(L)] + [prediction_tensor_dict[ith_outcome]], axis = 2)
        inputs_all_treat_3d = tf.cast(inputs_all_treat_3d, dtype = 'float32')
        inputs_all_control_3d = tf.cast(inputs_all_control_3d, dtype = 'float32')
        inputs_all_treat_3d_dict = {} 
        for l in range(L):
            inputs_all_treat_3d_l = tf.stack([x_basebid, x_sort_score,x_ecpm, x_cvr] + [w_all_treat if v == l else w_all_control for v in range(L)] + [prediction_tensor_dict[ith_outcome]], axis = 2)
            inputs_all_treat_3d_dict[l] = tf.cast(inputs_all_treat_3d_l, dtype = 'float32')
        inputs_all_treat_3d_dict = {} 
        for l in range(L):
            inputs_all_treat_3d_l = tf.stack([x_basebid, x_sort_score,x_ecpm, x_cvr] + [w_all_treat if v == l else w_all_control for v in range(L)] + [prediction_tensor_dict[ith_outcome]], axis = 2)
            inputs_all_treat_3d_dict[l] = tf.cast(inputs_all_treat_3d_l, dtype = 'float32')




        ## Modify the code 
        num_features= 4
        hfuncs_each_fold,  debias_terms_each_fold, probs_mat_each_fold, expos_mat_each_fold = {},{},{},{}
        eigen_value_each_fold = {} 
        history_each_fold = {}
        
        for f in range(n_folds):

            print("Start fold ", f )
            f_start, f_end = all_inds[f]

            xcov_train, xcov_test = train_test_split(x_covariates, all_inds, f)
            inputs_3d_train, inputs_3d_test = train_test_split(inputs_3d, all_inds, f)
            exposure_indicator_outcome_train, exposure_indicator_outcome_test = train_test_split(exposure_indicator_outcome, all_inds, f)
            inputs_all_treat_3d_train, inputs_all_treat_3d_test = train_test_split(inputs_all_treat_3d, all_inds, f)
            inputs_all_control_3d_train, inputs_all_control_3d_test = train_test_split(inputs_all_control_3d, all_inds, f)
            is_selected_indicator_train,is_selected_indicator_test = train_test_split(is_selected_indicator, all_inds, f)
            treat_control_dict = {} 
            for l in range(L):
                inputs_3d_train_l,inputs_3d_test_l  = train_test_split(inputs_all_treat_3d_dict[l], all_inds, f)

                treat_control_dict[l] = {'train':inputs_3d_train_l, 'test': inputs_3d_test_l}
            xcvr_train, xcvr_test =train_test_split(x_cvr, all_inds, f)
            myModelMultiple = model_memories[f]
            if ith_outcome == 1:
                history_f = myModelMultiple.fit(inputs_3d_train, exposure_indicator_outcome_train,epochs=1000,verbose=False)
                history_each_fold[f] = history_f 
                model_memories[f] = myModelMultiple
            else:
                myModelMultiple = model_memories[f]

            res_tempt = np.array(myModelMultiple.predict(inputs_all_treat_3d_test, verbose=False)) - np.array(myModelMultiple.predict(inputs_all_control_3d_test, verbose=False))
            exposure_indicator_array = is_selected_indicator_test
            model_pred_H = np.array(myModelMultiple.predict(inputs_3d_test, verbose=False))
            pred_H_new = np.array(myModelMultiple.predict(inputs_all_treat_3d_test, verbose=False)) - np.array(myModelMultiple.predict(inputs_all_control_3d_test, verbose= False))

            exposure_indicator_array = is_selected_indicator_test
            model_pred_all_treat = myModelMultiple.predict(inputs_all_treat_3d_test, verbose=False)
            model_pred_all_control = myModelMultiple.predict(inputs_all_control_3d_test, verbose=False)
            all_treat_array, all_control_array = np.array(model_pred_all_treat), np.array(model_pred_all_control)

            counterfactual_pred_dict = {} 
            for l in range(L):
                counterfactual_pred_dict[l] = myModelMultiple.predict(treat_control_dict[l]['test'], verbose=False)
            ## Outcome - prediction model 
            indicator_bool = tf.cast(is_selected_indicator_train, dtype=tf.bool)
            selected_elements = tf.boolean_mask(inputs_3d_train[:,:,:num_features], indicator_bool)
            x_prediction_train,x_prediction_test = train_test_split(prediction_tensor_dict[ith_outcome], all_inds, f)
            mus_T, mus_C  = x_prediction_test,x_prediction_test

            p_T, p_C  = all_treat_array[:, :K], all_control_array[:,:K]

            outcomes_train, outcomes_test = train_test_split(outcome_vector_dict[ith_outcome].values, all_inds, f)
            rewards_array = outcomes_test
            Ey1,Ey0 = np.sum(mus_T * p_T, axis = 1), np.sum(mus_C * p_C, axis = 1)
            pv1,pv0 = np.sum(exposure_indicator_array * p_T, axis = 1), np.sum(exposure_indicator_array * p_C, axis = 1)
            rewards_uv = np.sum(exposure_indicator_array * rewards_array, axis = 1 )

            treatment_indicator_array = 1 * (np.array(w_dict[ith_treat])[f_start:f_end,:])
            pv_given_uvw = p_T * treatment_indicator_array + p_C * (1 - treatment_indicator_array)
            ## FIX: get the realized probabilities 
            p_realized = model_pred_H[:,:K]

            
            ## 1. COMPUTE THE GRADIENT OF LOSSS  
            ## FIX: change to realized outcome 
            #dl1dtheta0 = pv_given_uvw - exposure_indicator_array
            dl1dtheta0 = p_realized - exposure_indicator_array
            dl1dtheta0 = dl1dtheta0[:, 1:] 
            ## FIX: iterate over all L 
            dl1dthetal_dict = {} 
            for l in range(L):
                treatment_indicator_array_l = w_dict[l][f_start:f_end, :]
                dl1dthetal_dict[l] = treatment_indicator_array_l *  (p_realized - exposure_indicator_array)
            dl2dmu = exposure_indicator_array * (mus_T -rewards_array)
            gradient_vector_l = np.concatenate([dl1dtheta0]+[dl1dthetal_dict[l] for l in range(L)] +[dl2dmu], axis =1 )


            




            ## 2. COMPUTE  THE GRADIENT OF H FUNCTION
            
            ## Need dictionary for each 
            
            dHdtheta0 = p_T * (mus_T - Ey1.reshape(mus_T.shape[0],1)) - p_C * (mus_C - Ey0.reshape(mus_C.shape[0],1))
            dHdtheta0 = dHdtheta0[:, 1:]



            ## FIX: iterate over each l 
            dHdthetal_dict = {} 
            for l in range(L):

                p_T_thetal = counterfactual_pred_dict[l][:,:K]
                Eyl = np.sum(mus_T * p_T_thetal, axis = 1)
                dHdthetal_dict[l] = p_T_thetal * (mus_T - Eyl.reshape(mus_T.shape[0],1))
                ## 0 for the groups that are not the target treatment group 
                if l != ith_treat:
                    dHdthetal_dict[l] = 0 * (p_T_thetal * (mus_T - Eyl.reshape(mus_T.shape[0],1)))

            #dHdthetal = p_T * (mus_T - Ey1.reshape(mus_T.shape[0],1))
            dHdmu = p_T - p_C
            #gradient_vector_H = np.concatenate([dHdtheta0,dHdthetal,dHdmu], axis =1 )

            ## FIX: iterate over all l 
            gradient_vector_H = np.concatenate([dHdtheta0]+[dHdthetal_dict[l] for l in range(L)]+[dHdmu], axis =1 )
            ## Gradient over all other treatments
            
            ## get gradient_vector_H for each category 
            dict_gradient_vector_H = {} 
            for c_group in cate_groups:
                J_indicator = (xcov_test == c_group) * 1 
                dict_gradient_vector_H[c_group] = compute_value_gradient_subgroup(p_T, mus_T, p_C, mus_C, J_indicator)


            ## 3. FIND THE EXPECTATION OF HESSIAN MATRIX 



            Hessian_all = np.zeros((inputs_3d_test.shape[0],(L+2) * K - 1,  (L+2) * K - 1))

            montecarlo_expected_probability = np.zeros(exposure_indicator_array.shape)

            selected_indicator_dict  = {}
            assignment_pd_dict = {} 
            dmu_dict = {} 



            M = 500
            for m in range(M):
                unique_qids = pd.DataFrame({"quota_id_": df_sellerside['quota_id_'].unique()})

                ## Need to modify here if multiple treatment 
                unique_qids['assignment'] = np.random.binomial(1, 0.5,unique_qids.shape[0])
                df_assignment_m = df_sellerside.loc[:,['quota_id_', 'req_id_','row_number']].merge(unique_qids, on = "quota_id_")
                W_matrix_m = df_assignment_m.pivot_table(index="req_id_", columns="row_number", values="assignment").values
                w_dict_m = {0:tf.convert_to_tensor(W_matrix_m == 1, dtype = float) }
                inputs_3d_m = tf.stack([x_basebid, x_sort_score,x_ecpm, x_cvr] + [tf.convert_to_tensor(W_matrix_m == 1, dtype = float)] + [prediction_tensor_dict[ith_outcome]], axis = 2)

                inputs_3d_test_m = inputs_3d_m[f_start:f_end,:]
                model_pred_m = np.array(myModelMultiple.predict(inputs_3d_test_m, verbose=False))[:,:K]
                outer_product_pv1pv2 = np.array([np.outer(row_[1:], row_[1:]) for row_ in model_pred_m])
                outer_product_treatment_indicator = np.array([np.outer(row_, row_) for row_ in model_pred_m])
                outer_product_pv1_one_minus_pv2 = np.array([np.outer(row_, 1-row_) for row_ in model_pred_m])



                is_selected_indicator_test = np.array(exposure_matrix[f_start:f_end,:])
                # selected_indicator_dict[m] = is_selected_indicator_test 
                d2l2dtheta0 = - np.array([np.outer(row_, row_) for row_ in model_pred_m])

                ## FIX: Iterate over l 
                d2l2dthetal_dict = {}
                for l in range(L):
                    ## K by K 

                    w_m_l = np.array(w_dict_m[l][f_start:f_end,:])

                    ## Off-diagonal terms 
                    d2l2dtheta1 =  -np.array([np.outer(row_, row_) for row_ in w_m_l]) * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                    # d2l2dtheta1 = - p_treat * p_treat * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                    ## Modify diagonal terms
                    for i in range(d2l2dtheta1.shape[0]):
                        treat_indicator_i = w_m_l[i,:]
                        probs_i = model_pred_m[i,:]
                        np.fill_diagonal(d2l2dtheta1[i],treat_indicator_i * probs_i * (1-probs_i))
                        # np.fill_diagonal(d2l2dtheta1[i], p_treat * probs_i * (1-probs_i))
                    d2l2dthetal_dict[l] = d2l2dtheta1


                ## FIX: iterate over all l1, l2 
                d2ldthetal1dthetal2 = {} 
                for l in range(L):
                    w_m_l = np.array(w_dict_m[l][f_start:f_end,:])
                    ## Off-diagonal terms 
                    ## !!!!!!!!!!!!!!!!! Ruohan: please double check the following, I modified the original one.
                    d2l2dtheta0dtheta1 = - np.multiply(w_m_l[:,np.newaxis, :], np.array([np.outer(row_, row_) for row_ in model_pred_m]))
                    # d2l2dtheta0dtheta1 = - p_treat * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                    for i in range(d2l2dtheta0dtheta1.shape[0]):
                        treat_indicator_i = w_m_l[i,:]
                        p_1minusp_i = model_pred_m[i,:] * (1 - model_pred_m[i,:])
                        np.fill_diagonal(d2l2dtheta0dtheta1[i], treat_indicator_i * p_1minusp_i)
                    ## NOTE: -1 to indicate the baseline theta 
                    d2ldthetal1dthetal2[(-1,l)] = d2l2dtheta0dtheta1[:,1:,:]
                    d2ldthetal1dthetal2[(l,-1)] = np.transpose(d2l2dtheta0dtheta1[:,1:,:], (0,2,1))
                    for l_prime in range(L):
                        if l != l_prime: 
                            w_m_l = np.array(w_dict_m[l][f_start:f_end,:])
                            w_m_l_prime = np.array(w_dict_m[l_prime][f_start:f_end,:])
                            indicator_outer = np.array([np.outer(w_m_l[i,:], w_m_l_prime[i,:]) for i in range(w_m_l.shape[0])])

                            d2l2dthetal1dthetal2 = -  indicator_outer * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                            # d2l2dthetal1dthetal2 = -  p_treat * p_treat * np.array([np.outer(row_, row_) for row_ in model_pred_m])
                            d2ldthetal1dthetal2[(l,l_prime)]  = d2l2dthetal1dthetal2
                            d2ldthetal1dthetal2[(l_prime,l)]  = np.transpose(d2l2dthetal1dthetal2, (0,2,1))
                        else:
                            d2ldthetal1dthetal2[(l,l)] = d2l2dthetal_dict[l]

                d2l2dmu = np.zeros(d2l2dtheta1.shape)

                for i in range(d2l2dmu.shape[0]):
                    #p_1minusp_i = (1 - model_pred_m[i,:]) * (1 - model_pred_m[i,:])
                    p_1minusp_i = model_pred_m[i,:] * (1 - model_pred_m[i,:])
                    # treatment_i = treatment_indicator_array[i,:]
                    # exposure_i = is_selected_indicator_test[i,:]
                    np.fill_diagonal(d2l2dtheta0[i], p_1minusp_i)
                    np.fill_diagonal(d2l2dmu[i], model_pred_m[i,:])


                d2l2dtheta0 = d2l2dtheta0[:,1:, 1:]
                # d2l2dtheta01_k_m_1_by_k = d2l2dtheta01[:, 1:,:]
                # d2l2dtheta10_k_by_k_m_1 = d2l2dtheta01[:, :,1:]
                Hessian_first_row = np.concatenate([d2l2dtheta0] + [d2ldthetal1dthetal2[(-1, l)] for l in range(L)] + [np.zeros((d2l2dtheta0.shape[0], K-1, K))], axis =2)

                ## 1 to L + 1 row 
                Hessian_middle_dict = {}
                for l in range(L):
                    row_l = np.concatenate([d2ldthetal1dthetal2[(l, -1)]] + [d2ldthetal1dthetal2[(l, l_prime)] for l_prime in range(L)] +[np.zeros((d2l2dtheta0.shape[0], K, K))], axis =2)

                    Hessian_middle_dict[l] = row_l                                                                           


                Hessian_third_row = np.concatenate((np.zeros((d2l2dtheta0.shape[0], K, K  * (L + 1 ) - 1 )), d2l2dmu), axis =2)

                Hessian = np.concatenate([Hessian_first_row] + [Hessian_middle_dict[l] for l in range(L)] + [Hessian_third_row], axis = 1 )

                dmu_dict[m] = d2l2dmu

                Hessian_all = Hessian_all + Hessian



            Hessian_final = Hessian_all / M
            count_finite = 0
            score_funcs = np.zeros(len(Hessian_final))
            count_infinite = 0

            for i in range(len(Hessian_final)):
                if is_invertible(Hessian_final[i]):
                    try:
                        score_funcs[i] = gradient_vector_H[i]@np.linalg.inv(Hessian_final[i])@gradient_vector_l[i]
                        count_finite += 1 
                    except: 
                        print("Fail for inversion")
                        count_infinite += 1



            ## END OF FOR LOOP FOR EACH ITERATION OVER CROSS FITTING
            hfuncs_f, debias_term_f = Ey1 - Ey0, score_funcs
            hfuncs_each_fold[f] =hfuncs_f
            debias_terms_each_fold[f] = debias_term_f


            ## Store data for NDCG evaluation 
            expos_mat_f = is_selected_indicator_test.astype(float)
            probs_mat_f = model_pred_H[:, :K]

            probs_mat_each_fold[f] = probs_mat_f 
            expos_mat_each_fold[f] = expos_mat_f 
            result_to_save = [hfuncs_each_fold, debias_terms_each_fold,probs_mat_each_fold,expos_mat_each_fold ]
            
            with open(f"prediction_result_new/outcome{ith_outcome}_{dh}_result.pickle", 'wb') as ff:
                pickle.dump( result_to_save, ff)
            
            ## SAVE CATE RESULTS
            
            for c_group in cate_groups:
                gradient_vector_H_cgroup = dict_gradient_vector_H[c_group]
                J_indicator = (xcov_test == c_group) * 1 
                Ey1_cgroup,Ey0_cgroup =  np.sum(mus_T * p_T * J_indicator, axis = 1), np.sum(mus_C * p_C * J_indicator, axis = 1)
                score_funcs_cgroup = np.zeros(len(Hessian_final))
                count_infinite = 0

                for i in range(len(Hessian_final)):
                    if is_invertible(Hessian_final[i]):
                        try:
                            score_funcs_cgroup[i] = gradient_vector_H_cgroup[i]@np.linalg.inv(Hessian_final[i])@gradient_vector_l[i]
                            count_finite += 1 
                        except: 
                            print("Fail for inversion")
                            count_infinite += 1
                hfuncs_f_cgroup, debias_term_f_cgroup = Ey1_cgroup - Ey0_cgroup, score_funcs_cgroup
            
                with open(f"prediction_result_new/outcome{ith_outcome}_{dh}_result_{c_group}_fold{f}.pickle", 'wb') as ff:
                    pickle.dump( [hfuncs_f_cgroup, debias_term_f_cgroup], ff)
